In [1]:
import matplotlib.pyplot as plt 
import metapack as mp
import pandas as pd
import numpy as np
import seaborn as sns
import xarray as xr
%matplotlib inline

In [2]:
pkg = mp.open_package('http://library.metatab.org/ceden.waterboards.ca.gov-beachwatch-sandiego-2.zip')

pkg

df = pkg.resource('beachwatch-sd').read_csv(parse_dates=True)

# It looks like the prefix of the station code groups stations, maybe into watersheds. 
df['stationgroup'] = df.stationcode.str[:2]

df = df[['stationcode','sampledate','methodname','unit','analyte','result']]


In [3]:
# Group stats

dfc_gs = df[['stationcode','sampledate','methodname','analyte','unit', 'result']]\
         .groupby(['stationcode','methodname','analyte','unit']).agg({'result':['count','std','mean','median']})
dfc_gs.columns = ['result_count', 'group_std', 'group_mean', 'group_median']
#dfm.reset_index(inplace=True)
dfc_gs.head()

result_count    group_std  \
stationcode methodname  analyte         unit                                    
EH-010      Colilert-18 Coliform, Total MPN/100 mL            16  2754.504235   
                        E. coli         MPN/100 mL            16   414.620791   
            Enterolert  Enterococcus    MPN/100 mL           389   426.067900   
                                        cfu/100mL              9   681.020191   
            MTF         Coliform, Fecal MPN/100 mL           213  1647.469843   

                                                     group_mean  group_median  
stationcode methodname  analyte         unit                                   
EH-010      Colilert-18 Coliform, Total MPN/100 mL  2069.125000         524.0  
                        E. coli         MPN/100 mL   190.500000          20.0  
            Enterolert  Enterococcus    MPN/100 mL   151.406170          10.0  
                                        cfu/100mL    320.333333          10.0  
            MTF         Coliform, Fecal MPN/100 mL   376.478873          20.0

In [4]:
_1 = df.set_index(['stationcode','methodname','analyte','unit']).join(dfc_gs).reset_index()

# Mark the records that are in excess of the median and mean,
_1['gt_median'] = _1.result > _1.group_median
_1['gt_mean']   = _1.result > _1.group_mean

# However, we still have an issue with there being multiple measurements per day/analyte/method/station. 
# The gt_mean and gt_median are now linked to the mean and median for the day/analyte/method/station, so let's just
# consider that normalization enough and average the flags. Since they are binary, they will  convert to 0/1
_2 = _1.reset_index().set_index('sampledate').groupby(['stationcode','methodname','analyte','unit']).resample('1d').mean().reset_index()
dfc = _2
dfc.head()


,stationcode,methodname,analyte,unit,sampledate,index,result,result_count,group_std,group_mean,group_median,gt_median,gt_mean
0,EH-010,Colilert-18,"Coliform, Total",MPN/100 mL,2005-09-16,14.0,86.0,16.0,2754.504235,2069.125,524.0,0.0,0.0
1,EH-010,Colilert-18,"Coliform, Total",MPN/100 mL,2005-09-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,EH-010,Colilert-18,"Coliform, Total",MPN/100 mL,2005-09-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,EH-010,Colilert-18,"Coliform, Total",MPN/100 mL,2005-09-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,EH-010,Colilert-18,"Coliform, Total",MPN/100 mL,2005-09-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Proportion of gt_mean and mt_median that are different. 
len(dfc[ dfc.gt_median.notnull() & dfc.gt_mean.notnull() &  (dfc.gt_median != dfc.gt_mean)])  / len(dfc)

0.013614282528397813

In [6]:
_ = dfc[dfc.gt_median.notnull()]
len(_[_.gt_median> 0.5]) / len(_)

0.3119946815311054

In [7]:
_ = dfc[dfc.gt_mean.notnull()]
len(_[_.gt_mean> 0.5]) / len(_)

0.11064972257025237

# Merge in environmental data

In [8]:
env_pkg =  mp.open_package('http://library.metatab.org/sandiegodata.org-water_quality-3.zip')

In [9]:
trr = env_pkg.resource('tides_river_rain').read_csv(parse_dates=True)
trr.head()

,date,discharge_rate,dailyprecip,wl_max,wl_min
0,1988-01-01,22.0,NaN,NaN,NaN
1,1988-01-02,20.0,NaN,NaN,NaN
2,1988-01-03,18.0,NaN,NaN,NaN
3,1988-01-04,18.0,NaN,NaN,NaN
4,1988-01-05,21.0,NaN,NaN,NaN


In [ ]:
_1 = trr.set_index('date').resample('1d').first()
dfc_env = dfc.join(_1)
dfc_env.head()

In [ ]:
fig, ax = plt.subplots(figsize=(15,.5*len(dfc_env.columns)))
sns.heatmap(dfc_env.isnull().T,cbar=False,xticklabels=False,cmap = 'viridis', ax=ax );

In [ ]:
dfc_env.dailyprecip.dropna().index.min()

In [ ]:
dfc_env.discharge_rate.median(), dfc_env.discharge_rate.mean()

In [ ]:
dfc_env['discharge_gt_mean']   = dfc_env.discharge_rate > dfc_env.discharge_rate.mean()
dfc_env['discharge_gt_median'] = dfc_env.discharge_rate > dfc_env.discharge_rate.median()

In [ ]:
#pd.crosstab(dfc_env.discharge_gt_mean,dfc_env.gt_median).plot(kind='bar');